In [1]:
%aiida
from pymatgen.io.cif import CifParser
import pymongo
from collections import OrderedDict as odict

import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s',
                    level=logging.DEBUG,
                    filename='structureREST.log')

In [2]:
DB = 'structureREST'
COLLECTIONS = ['mpds', 'icsd', 'cod']

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client[DB]

In [3]:
cod = Group.get_from_string('cod_cif_raw')
mpds = Group.get_from_string('mpds_cif_raw')
icsd = Group.get_from_string('icsd_cif_raw')

In [43]:
N = 10000
for group_name, group in {'cod': cod,
                          'icsd': icsd,
                          'mpds': mpds}.items():
    logging.info('Creating collection %s', group_name)
    collection = db[group_name]
    documents = []
    for cif in group.nodes:
        path, structures, source, structure, document = [None] * 5
        path = cif.get_file_abs_path()
        try:
            structures = CifParser(path).get_structures(primitive=False)
        except Exception as exception:
            logging.error('%s', str(exception))
        else:
            source = cif.get_attr('source')
            for structure in structures:
                document = {
                    'source': source,
                    'structure': structure.as_dict()
                }
                documents.append(document)
                if len(documents) == N:
                    print '%s' % str(sys.getsizeof(documents))
                    logging.debug('Inserting %d documents into %s',
                                  len(documents), group_name)
                    try:
                        result = collection.insert_many(documents)
                    except Exception as exception:
                        logging.critical('%s', str(exception))
                        raise Exception(exception)
                    else:
                        logging.info('Successfuly inserted %d documents into %s',
                                    len(result.inserted_ids), group_name)
                        documents = []
    
    if documents:
        logging.info('Inserting %d documents into %s', len(documents), group_name)
        try:
            result = collection.insert_many(documents)
        except Exception as exception:
            logging.critical('%s', str(exception))
            raise Exception(exception)
        else:
            logging.info('Successfuly inserted %d documents into %s',
                         len(result.inserted_ids), group_name)
            documents = []

104
CPU times: user 3.84 s, sys: 248 ms, total: 4.09 s
Wall time: 4.32 s
200
CPU times: user 3.98 s, sys: 313 ms, total: 4.29 s
Wall time: 4.48 s
920
CPU times: user 7.59 s, sys: 289 ms, total: 7.88 s
Wall time: 8.19 s
4280
CPU times: user 21.3 s, sys: 410 ms, total: 21.7 s
Wall time: 22.4 s
9032
CPU times: user 34.6 s, sys: 694 ms, total: 35.3 s
Wall time: 36.5 s
43048
CPU times: user 2min 47s, sys: 2.98 s, total: 2min 50s
Wall time: 2min 57s
87632
CPU times: user 5min 29s, sys: 6.58 s, total: 5min 35s
Wall time: 5min 52s
